# Detect AI vs Human generated images
In this notebook, I aim to build a convnet architecture for detecting real images from fake ones.

In [1]:
# All imports needed for this classification project
import kagglehub
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import ResNet152, Xception
from tensorflow.keras.applications.resnet import preprocess_input

In [2]:
# Global variables 
IMG_SIZE = 224

# Reproducability
def set_seed(seed=44):
    np.random.seed(seed)
    tf.random.set_seed(seed)
set_seed()

In [3]:
# Download latest version
data_path = kagglehub.dataset_download("alessandrasala79/ai-vs-human-generated-dataset")

print("Path to dataset files:", data_path)

Path to dataset files: /kaggle/input/ai-vs-human-generated-dataset


In [4]:
# Csv files
train = pd.read_csv(os.path.join(data_path,"train.csv"), index_col=0)
test = pd.read_csv(os.path.join(data_path, "test.csv"))

In [5]:
print(train.head())

                                         file_name  label
0  train_data/a6dcb93f596a43249135678dfcfc17ea.jpg      1
1  train_data/041be3153810433ab146bc97d5af505c.jpg      0
2  train_data/615df26ce9494e5db2f70e57ce7a3a4f.jpg      1
3  train_data/8542fe161d9147be8e835e50c0de39cd.jpg      0
4  train_data/5d81fa12bc3b4cea8c94a6700a477cf2.jpg      1


In [6]:
# Create full path to the images
train["file_name"] = train["file_name"].apply(lambda x: os.path.join(data_path, x))
test["id"] = test["id"].apply(lambda x: os.path.join(data_path, x))

In [7]:
# Split train into train and validation data
train_ds, val_ds = train_test_split(train, test_size=0.2, random_state=44)
train_ds = tf.data.Dataset.from_tensor_slices((train_ds["file_name"].tolist(), train_ds["label"]))
val_ds = tf.data.Dataset.from_tensor_slices((val_ds["file_name"].tolist(), val_ds["label"]))

In [8]:
def load_resize_and_rescale(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    image = (image / 255.0)
    return image, label

In [9]:
def augment(image_label, seed):
  image, label = image_label
  image, label = load_resize_and_rescale(image, label)
  image = tf.image.resize_with_crop_or_pad(image, IMG_SIZE + 6, IMG_SIZE + 6)
  # Make a new seed.
  new_seed = tf.random.split(seed, num=1)[0, :]
  # Random crop back to the original size.
  image = tf.image.stateless_random_crop(
      image, size=[IMG_SIZE, IMG_SIZE, 3], seed=seed)
  # Random brightness.
  image = tf.image.stateless_random_brightness(
      image, max_delta=0.5, seed=new_seed)
  image = tf.clip_by_value(image, 0, 1)
  return image, label

In [10]:
# Create a `Counter` object and `Dataset.zip` it together with the training set.
counter = tf.data.experimental.Counter()
train_ds = tf.data.Dataset.zip((train_ds, (counter, counter)))
batch_size = 32

In [11]:
train_ds = (
    train_ds
    .shuffle(1000)
    .map(augment, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)

In [12]:
val_ds = (
    val_ds
    .map(load_resize_and_rescale, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)

In [ ]:
pretrained_base = ResNet152(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
avg = keras.layers.GlobalAveragePooling2D()(pretrained_base.output)
output = keras.layers.Dense(2, activation="softmax")(avg)
model = keras.models.Model(inputs=pretrained_base.input, outputs=output)

In [ ]:
for layer in pretrained_base.layers:
    layer.trainable = False

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam",
              metrics=["accuracy"])
history = model.fit(train_ds,
                    steps_per_epoch=int(0.75 * len(train_ds) / batch_size),
                    validation_data=val_ds,
                    validation_steps=int(0.15 * len(train_ds) / batch_size),
                    epochs=5, 
                    verbose=1)

In [ ]:
for layer in pretrained_base.layers:
    layer.trainable = True

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam",
              metrics=["accuracy"])
history = model.fit(train_ds,
                    steps_per_epoch=int(0.75 * len(train_ds) / batch_size),
                    validation_data=val_ds,
                    validation_steps=int(0.15 * len(train_ds) / batch_size),
                    epochs=5, 
                    verbose=1)

In [ ]:
history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['accuracy', 'val_accuracy']].plot();

In [ ]:
test_data = tf.data.Dataset.from_tensor_slices(test_full_path)
test_data = test_data.map(load_preprocess_image_1, num_parallel_calls=tf.data.AUTOTUNE)
test_data = test_data.batch(batch_size)

In [ ]:
pred = model.predict(test_data)

In [ ]:
ar = [0 if i > 0.5 else 1 for i,j in pred]

In [ ]:
test["label"] = ar
test.to_csv("predictions.csv", index=False)